In [1]:
import pandas as pd

In [11]:
df_news = pd.read_csv("Reliance_sentiment.csv")

In [3]:
df_price = pd.read_csv("reliance_prices.csv")

In [12]:
df_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384 entries, 0 to 1383
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Date         1384 non-null   object 
 1   Open         1384 non-null   float64
 2   High         1384 non-null   float64
 3   Low          1384 non-null   float64
 4   Close        1384 non-null   float64
 5   Adj Close    1384 non-null   float64
 6   Volume       1384 non-null   int64  
 7   SMA3         1384 non-null   float64
 8   SMA5         1384 non-null   float64
 9   SMA9         1384 non-null   float64
 10  SMA15        1384 non-null   float64
 11  SMA30        1384 non-null   float64
 12  EMA3         1384 non-null   float64
 13  EMA5         1384 non-null   float64
 14  EMA9         1384 non-null   float64
 15  EMA15        1384 non-null   float64
 16  EMA30        1384 non-null   float64
 17  1d_diff      1384 non-null   float64
 18  5d_diff      1384 non-null   float64
 19  10d_di

In [13]:
df_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 654 entries, 0 to 653
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   date              654 non-null    object 
 1   title_senti_comp  654 non-null    float64
 2   news_senti_comp   654 non-null    float64
 3   avg_senti_comp    654 non-null    float64
dtypes: float64(3), object(1)
memory usage: 20.6+ KB


In [15]:
df_combined = pd.merge(left=df_price,right=df_news,how="left",left_on = 'Date',right_on='date')

In [18]:
df_combined.drop(['date'],axis=1,inplace=True)

In [19]:
df_combined.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,SMA3,SMA5,SMA9,...,EMA9,EMA15,EMA30,1d_diff,5d_diff,10d_diff,Volume_100k,title_senti_comp,news_senti_comp,avg_senti_comp
1379,2022-07-25,2467.449951,2467.449951,2403.000000,2420.399902,2420.399902,10665470,2469.933350,2469.980029,2438.738905,...,2450.729008,2453.819271,2484.103313,-82.700195,-1.850098,-3.500000,106.655,NaN,NaN,NaN
1380,2022-07-26,2421.100098,2443.899902,2411.350098,2421.500000,2421.500000,5216466,2448.333333,2466.860010,2443.622233,...,2444.883206,2449.779362,2480.064390,1.100098,-15.600098,1.050049,52.165,NaN,NaN,NaN
1381,2022-07-27,2419.949951,2427.000000,2402.000000,2419.199951,2419.199951,3994321,2420.366618,2450.100000,2446.072238,...,2439.746555,2445.956936,2476.137652,-2.300049,-83.800049,41.649902,39.943,NaN,NaN,NaN
1382,2022-07-28,2436.000000,2467.949951,2421.000000,2457.100098,2457.100098,5766936,2432.600016,2444.260010,2452.216688,...,2443.217264,2447.349831,2474.909423,37.900146,-29.199951,59.950195,57.669,NaN,NaN,NaN
1383,2022-07-29,2474.699951,2517.100098,2464.000000,2509.449951,2509.449951,6982808,2461.916667,2445.529980,2461.905572,...,2456.463801,2455.112346,2477.137844,52.349854,6.349854,107.649902,69.828,NaN,NaN,NaN


In [20]:
df_combined.isna().sum()

Date                  0
Open                  0
High                  0
Low                   0
Close                 0
Adj Close             0
Volume                0
SMA3                  0
SMA5                  0
SMA9                  0
SMA15                 0
SMA30                 0
EMA3                  0
EMA5                  0
EMA9                  0
EMA15                 0
EMA30                 0
1d_diff               0
5d_diff               0
10d_diff              0
Volume_100k           0
title_senti_comp    845
news_senti_comp     845
avg_senti_comp      845
dtype: int64